# Conformance Checking in Declare4Py

This tutorial explains how to perform the checking of a DECLARE model and how to browse the results.

After importing the Declare4Py package and specified the paths of the log and of the DECLARE model, a `Declare4Py` object has to be instantiated.

In [ ]:
import os
from declare4py.declare4py import Declare4Py


log_path = os.path.join("..", "tests", "Sepsis Cases.xes.gz")
model_path = os.path.join("..", "tests", "declare_models", "data_model.decl")

d4py = Declare4Py()

The DECLARE constraints supported by Declare4Py can be retrieved with the `get_supported_templates()` function:

In [ ]:
print(d4py.get_supported_templates())

Notice that for the templates `Existence`, `Absence` and `Exactly` an additional parameter is necessary for the cardinality. This has to be encoded in the DECLARE `.decl` model with a numeric suffix, for example `Exactly2` or `Existence23`.

The next step is the parsing of the log and of the DECLARE model.

In [ ]:
d4py.parse_xes_log(log_path)
d4py.parse_decl_model(model_path)

The DECLARE model can be inspected by getting all the activity names or the constraints. This information is returned as a list of strings.

In [ ]:
model_activities = d4py.get_model_activities()
model_constraints = d4py.get_model_constraints()
for idx, act in enumerate(model_activities):
    print(idx, act)
print("-----------------")
for idx, constr in enumerate(model_constraints):
    print(idx, constr)

Once the log and the DECLARE model are loaded, the `conformance_checking()` function will perform the model checking. The input boolean parameter `consider_vacuity=true` means that vacuously satisfied traces are considered as satisfied, violated otherwise. This function returns a Python dictionary containing the results indexed by trace in the log. Each key is a tuple containing the trace id and its name (that can be retrieved with `get_trace_keys()`). The value is a Python dictionary with keys the names of the constraints (that can be retrieved with `model_constraints()`) and values a `CheckerResult` object containing the number of pendings, activations, violations, fulfilments and the truth value of the trace for that constraint.

```
model_check_res = {trace_1: {constr_1: CheckerResult object, constr_2: CheckerResult object, ...},
                   trace_2: {constr_1: CheckerResult object, ... },
                    ...
                  }
```
The `CheckerResult` objects can be accessed by the attributes `num_pendings`, `num_activations`, `num_fulfillments`, `num_violations` and `state`.

In [ ]:
model_check_res = d4py.conformance_checking(consider_vacuity=False)

Let's inspect the results for the trace `(1, 'B')` and the constraint
`Chain Response[LacticAcid, Leucocytes] |A.LacticAcid <= 0.8 |T.Leucocytes >= 13.8 |0,2778,m`

In [ ]:
constr_id = 11
trace_id = (1, 'B')
print(f"Constraint to check: {model_constraints[constr_id]}")
print(f"Number of pendings: {model_check_res[trace_id][model_constraints[constr_id]].num_pendings}")
print(f"Number of activations: {model_check_res[trace_id][model_constraints[constr_id]].num_activations}")
print(f"Number of fulfilments: {model_check_res[trace_id][model_constraints[constr_id]].num_fulfillments}")
print(f"Number of violation: {model_check_res[trace_id][model_constraints[constr_id]].num_violations}")
print(f"Truth value: {model_check_res[trace_id][model_constraints[constr_id]].state}")

The checker results can be printed with the `print_conformance_results` function.